In [1]:
library(AnnotationGx)
library(data.table)
library(httr)
library(jsonlite)

In [2]:
ccle_metadata <- qs::qread("/home/bioinf/bhklab/jermiah/psets/PharmacoSet-Pipelines/CCLE/procdata/preprocessedMetadata.qs")

rdataPath <- "/home/bioinf/bhklab/jermiah/psets/PharmacoSet-Pipelines/CCLE/.RData"

load(rdataPath)

In [3]:
head(treatmentMetadata[, .(PubChem.CID, CCLE.cleanedTreatmentName)])

PubChem.CID,CCLE.cleanedTreatmentName
<int>,<chr>
36314,PACLITAXEL
60700,TOPOTECAN
60838,IRINOTECAN
176870,ERLOTINIB
208908,LAPATINIB
216239,SORAFENIB


In [4]:
result <- getPubChemAnnotation(compound = treatmentMetadata$PubChem.CID[1])

In [5]:
result

$cid
[1] 36314

$`ChEMBL ID`
[1] "Compound::CHEMBL428647"

In [6]:
# merged$cids[1:10]

annotations <- c('ChEMBL ID', 'NSC Number', 'DILI', 'CAS', 'ATC Code')
# parallelRun<- BiocParallel::MulticoreParam(workers=8, progressbar=TRUE, stop.on.error=FALSE)

result <- 
    suppressWarnings(rbindlist(
        BiocParallel::bptry(
                BiocParallel::bplapply(
            c(merged$cids[1:50]), function(y){
                BiocParallel::bptry(
                    annotationCID <- BiocParallel::bplapply(annotations, function(x) {
                        as.data.table(getPubChemCHEMBL(y, type = x))
                        },
                        BPPARAM = BiocParallel::MulticoreParam(workers = 5, stop.on.error=FALSE)
                    )
                )
                Reduce(function(x, y) merge(x, y, allow.cartesian = TRUE), annotationCID)
            },
            BPPARAM = BiocParallel::MulticoreParam(workers = 6, progressbar = TRUE, stop.on.error=FALSE)
        )
    ),
    fill = TRUE,
))
# save result to RDS file 
saveRDS(result, "ctrp_pubchem_annotations.RDS")

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'X' in selecting a method for function 'bplapply': object 'merged' not found


In [ ]:
result

In [ ]:
test <- merged[540:545, c("cpd_name","cids")]


test



In [ ]:
subs[lapl]

In [ ]:
# get all paths in metadata folder
metadata_paths <- list.files(path = "metadata", pattern = "*.RDS", full.names = TRUE)
print(metadata_paths)

In [ ]:
# read in 2-9
atc <- as.data.table(readRDS(metadata_paths[2]))
parsed_atc<- AnnotationGx:::.parseATCannotations(atc)
print(dim(parsed_atc))
parsed_atc[1:5]

In [ ]:
names(dili)

In [ ]:
dili$Data